In [4]:
import numpy as np # algebra linear
import pandas as pd # ler os dados
import matplotlib.pyplot as plt # mostrar os dados

In [9]:
data = pd.read_csv('/kaggle/input/digit-recognizer/train.csv') #dados

In [10]:
data.head() #teste

In [12]:
#dividir os dados entre dados de treino e do programa
data = np.array(data)
m,n = data.shape
np.random.shuffle(data)

data_dev = data[0:1500].T
Y_dev = data_dev[0]
X_dev = data_dev[1:n]

data_train = data[1500:m].T
Y_train = data_train[0]
X_train = data_train[1:n]




In [13]:
Y_train #teste

In [14]:
X_train[:, 0].shape # conferindo se tem os 784 pixels

In [17]:
#inserindo primeiros parametros
def init_params():
    W1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    return W1, b1, W2, b2

        #ReLu(x) = x if x>0 && 0 if x <= 0
    
def ReLU(Z):
        return np.maximum(Z, 0)
    
        #softmax = eᶻᶦ/Σᵏ eᶻʲ
                    #  ʲ⁼¹
    
def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A
    
        # propagação pra frente:
        #𝑍^[1] =𝑊^[1] 𝑋+𝑏^[1]
        #𝐴^[1] =𝑔ReLU(𝑍^[1]))
        #𝑍^[2] =𝑊^[2] 𝐴^[1] +𝑏^[2]
        #𝐴^[2] =𝑔softmax(𝑍^[2])
    
def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

    # pega a derivada da função de ativação linear retificada 
    #por ser uma linha reta se um elemento for maior que 0 vai ser true e vai retornar 1)
    
def ReLU_deriv(Z):
    return Z > 0

    #faz uma matriz de dados Y com apenas uma coluna

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

    #propagação pra trás 
    #𝑑𝑍^[2] =𝐴^[2] −𝑌
    #𝑑𝑊^[2] =1/𝑚𝑑 . 𝑍^[2] 𝐴^[1]ᵀ
    #𝑑𝐵^[2] =1/𝑚 . Σ𝑑𝑍^[2]
    #𝑑𝑍^[1] =𝑊^[2]ᵀ 𝑑𝑍^[2].∗𝑔^[1]′ (𝑧^[1])
    #𝑑𝑊^[1] =1/𝑚𝑑 .𝑍^[1] 𝐴[0]ᵀ
    #𝑑𝐵^[1] =1/𝑚 Σ𝑑𝑍^[1]

def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

        #atualiza os parametros

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2    
    return W1, b1, W2, b2

In [21]:
#pega os palpites
def get_predictions(A2):
    return np.argmax(A2, 0)

    #pega a precisão

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

#otimiza o codigo utilizando gradiente para que cada passo corresponda a direção de declive máximo e encontrando o valor mínimo da nossa função

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2

In [22]:
#treino
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.10, 500)

98.% de precisão